# Forecasting Models

## Objectives
- Establish strong forecasting baselines
- Train a deep learning forecasting model (PyTorch)
- Compare models using clear metrics

This notebook builds on the preprocessing logic from Notebook 02.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

plt.rcParams['figure.figsize'] = (14, 5)

## Load and Prepare Data

To keep the notebook self-contained, we repeat minimal preprocessing steps.

In [2]:
from pathlib import Path

DATA_PATH = Path('../data/raw/LD2011_2014.txt')
df = pd.read_csv(DATA_PATH, sep=';', index_col=0, parse_dates=True, decimal=',')

ts = df[df.columns[0]].rename('load')

n = len(ts)
train_ts = ts.iloc[:int(0.7*n)]
val_ts = ts.iloc[int(0.7*n):int(0.85*n)]

mean, std = train_ts.mean(), train_ts.std()
scale = lambda s: (s - mean) / std

train_ts = scale(train_ts)
val_ts = scale(val_ts)

## Sliding Window Construction

In [3]:
def create_windows(series, input_len, horizon):
    X, y = [], []
    values = series.values
    for i in range(len(values) - input_len - horizon + 1):
        X.append(values[i:i+input_len])
        y.append(values[i+input_len:i+input_len+horizon])
    return np.array(X), np.array(y)

INPUT_LEN = 24 * 7
HORIZON = 24

X_train, y_train = create_windows(train_ts, INPUT_LEN, HORIZON)
X_val, y_val = create_windows(val_ts, INPUT_LEN, HORIZON)

## Baseline Model: Seasonal Naive

In [4]:
def seasonal_naive_forecast(X, season=24):
    return X[:, -season:]

y_pred_naive = seasonal_naive_forecast(X_val)

def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

naive_mae = mae(y_val, y_pred_naive)
naive_mae

np.float64(0.3546049412385093)

## Deep Learning Model (N-BEATS-style MLP)

In [6]:
class SimpleNBeats(nn.Module):
    def __init__(self, input_len, horizon):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_len, 256), nn.ReLU(),
            nn.Linear(256, 256), nn.ReLU(),
            nn.Linear(256, horizon)
        )

    def forward(self, x):
        return self.net(x)

model = SimpleNBeats(INPUT_LEN, HORIZON)

## Training Loop

In [7]:
train_loader = DataLoader(
    list(zip(X_train, y_train)), batch_size=64, shuffle=True
)
val_loader = DataLoader(
    list(zip(X_val, y_val)), batch_size=64
)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

for epoch in range(5):
    model.train()
    total_loss = 0
    for xb, yb in train_loader:
        xb = torch.tensor(xb, dtype=torch.float32)
        yb = torch.tensor(yb, dtype=torch.float32)
        optimizer.zero_grad()
        preds = model(xb)
        loss = loss_fn(preds, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoch {epoch+1}, Train Loss: {total_loss/len(train_loader):.4f}')

/tmp/ipykernel_9920/2035466745.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  xb = torch.tensor(xb, dtype=torch.float32)
/tmp/ipykernel_9920/2035466745.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  yb = torch.tensor(yb, dtype=torch.float32)


Epoch 1, Train Loss: 0.2972
Epoch 2, Train Loss: 0.2745
Epoch 3, Train Loss: 0.2624
Epoch 4, Train Loss: 0.2483
Epoch 5, Train Loss: 0.2360


## Evaluation

In [8]:
model.eval()
with torch.no_grad():
    preds = []
    for xb, _ in val_loader:
        xb = torch.tensor(xb, dtype=torch.float32)
        preds.append(model(xb).numpy())

preds = np.vstack(preds)

dl_mae = mae(y_val[:len(preds)], preds)

print('Seasonal Naive MAE:', naive_mae)
print('Deep Learning MAE:', dl_mae)

/tmp/ipykernel_9920/2239188498.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  xb = torch.tensor(xb, dtype=torch.float32)


Seasonal Naive MAE: 0.3546049412385093
Deep Learning MAE: 0.3006225295616382


## Summary

- Seasonal Naive provides a strong baseline
- A simple deep learning model already improves performance
- This setup enables residual-based anomaly detection in the next notebook